<a href="https://colab.research.google.com/github/solomontessema/building-ai-agents/blob/main/notebooks/5.4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table>
  <tr>
    <td><img src="https://ionnova.com/img/ionnova_logo_name_2.png" width="120px"></td>
    <td><h1>Connecting Retrieval with Generation</h1></td>
  </tr>
</table>

In [ ]:
!pip install -qU langchain==1.1.0 langchain-openai==1.1.0 langchain-community==0.4.1 faiss-cpu==1.13.2 python-dotenv==1.1.1

In [ ]:
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from dotenv import load_dotenv
import requests

load_dotenv()

# Download file from GitHub
GITHUB_RAW_URL = (
    "https://raw.githubusercontent.com/"
    "solomontessema/building-ai-agents/main/assets/langgraph_intro.txt"
)

response = requests.get(GITHUB_RAW_URL)
response.raise_for_status()

# Create the Index (The "Save" Step)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
docs = [Document(page_content=response.text)]
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)

vectorstore = FAISS.from_documents(chunks, embeddings)
vectorstore.save_local("faiss_index")
print("Vector store saved successfully!")

# Load the Index (The "Load" Step)
vectorstore = FAISS.load_local(
    "faiss_index",
    embeddings,
    allow_dangerous_deserialization=True
)
retriever = vectorstore.as_retriever()

# The Chain
prompt = PromptTemplate.from_template("Answer based on context: {context}\n\nQuestion: {question}")
llm = ChatOpenAI(model="gpt-4o", temperature=0)

chain = (
    {"context": retriever | (lambda docs: "\n\n".join(d.page_content for d in docs)),
     "question": RunnablePassthrough()}
    | prompt | llm | StrOutputParser()
)

print("Response:", chain.invoke("What is LangGraph?"))
